In [ ]:
#Cell 01
# -*- coding: utf-8 -*-
"""FlaskAPI.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1UMpXNXfIb5n5LGXT0x55P50zATDDlpSY
"""
!pip install pyngrok
!ngrok config add-authtoken 2vIiU1cclMLZEWBb3FgGNuzmK1V_6ehWcSCgvAqnDTXrvsrZn

from pyngrok import ngrok

# Kill all existing tunnels
ngrok.kill()

# Cell 1: Imports and Configuration
import requests
import json
from IPython.display import clear_output

# Install required packages
!pip install sentence-transformers pinecone-client transformers requests numpy flask-ngrok pyngrok
!pip install pinecone

from flask import Flask, request, jsonify, send_from_directory
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import json
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import random
from IPython.display import HTML, display
from google.colab import output
import os
import threading

from pyngrok import ngrok


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/installer.py", line 123, in install_ngrok
    download_path = _download_file(url, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/installer.py", line 316, in _download_file
    raise e
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/installer.py", line 274, in _download_file
    response = urlopen(url, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
#Cell 02
# GitHub Configuration - UPDATE THESE!
GITHUB_USER = "Naduni13"
REPO_NAME = "travel-data"
BRANCH = "main"  # or your branch name

# File names (keep these unless you renamed your files)
DATA_FILES = {
    "accommodation": "accommodation.json",
    "attractions": "attractions.json",
    "restaurants": "restaurants.json",
    "activities": "activities.json",
    "transport": "transport.json",
    "guides": "guides.json"
}

# Cell 2: Data Loading Function
def load_github_data():
    base_url = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/"
    data_urls = {name: base_url + filename for name, filename in DATA_FILES.items()}

    loaded_data = {}
    print("Loading data from GitHub...")

    for name, url in data_urls.items():
        try:
            response = requests.get(url)
            response.raise_for_status()
            loaded_data[name] = response.json()
            print(f"✓ {name} data loaded")
        except Exception as e:
            print(f"✗ Error loading {name}: {str(e)}")
            loaded_data[name] = []

    clear_output(wait=True)
    print("All data loaded successfully!\n")
    return (loaded_data["attractions"], loaded_data["accommodation"],
            loaded_data["restaurants"], loaded_data["activities"],
            loaded_data["transport"], loaded_data["guides"])

# Cell 3: Load and Verify Data
activity_data, hotels_data, restaurants_data, activity_data_new, transport_data, guides_data = load_github_data()

# Verification
print(f"Hotels loaded: {len(hotels_data)}")
print(f"Restaurants loaded: {len(restaurants_data)}")
print(f"Attractions loaded: {len(activity_data)}")
print(f"Activities loaded: {len(activity_data_new)}")
print(f"Transport options: {len(transport_data)}")
print(f"Guides available: {len(guides_data)}")

# # Print sample guide data if available
# if guides_data:
#     print("\nSample Guide:")
#     print(f"Name: {guides_data[0].get('fullName', 'N/A')}")
#     print(f"Languages: {', '.join(guides_data[0].get('languagesSpoken', []))}")
#     print(f"Price per day: LKR {guides_data[0].get('pricePerDay', 0)}")
#     print(f"Contact: {guides_data[0].get('contactNumber', 'N/A')}")
# else:
#     print("\nNo guides data loaded")

All data loaded successfully!

Hotels loaded: 59
Restaurants loaded: 150
Attractions loaded: 47
Activities loaded: 289
Transport options: 47
Guides available: 30


In [ ]:
#Cell 03
public_url = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print(f"App is live at: {public_url.public_url}")

ERROR:pyngrok.process.ngrok:t=2025-04-08T06:26:49+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-08T06:26:49+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
#Cell 04
# -*- coding: utf-8 -*-
"""Travel Itinerary Planner with Server Storage and New Tab Interface"""

# Initialize Flask app
app = Flask(__name__, static_folder='static')
run_with_ngrok(app)  # This makes the app available via ngrok

@app.after_request
def add_ngrok_header(response):
    response.headers["ngrok-skip-browser-warning"] = "true"
    return response

# Global variables to store data and model
global_data = {}
model = None
pc = None


In [ ]:
#Cell 05
@app.route('/')
def home():
    return """
    <!DOCTYPE html>
<html>
<head>
    <title>Travel Itinerary Planner</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <style>
        :root {
            --primary-color: #1f9c9d;
            --primary-light: #e0f2f2;
            --primary-dark: #168586;
            --accent-color: #FFAA1D;
            --accent-light: #ffd58f;
            --dark-color: #2c3e50;
            --light-color: #f8f9fa;
            --white: #ffffff;
            --text-color: #333;
            --border-radius: 12px;
            --box-shadow: 0 4px 20px rgba(0,0,0,0.1);
            --transition: all 0.3s cubic-bezier(0.25, 0.8, 0.25, 1);
        }

        * {
            box-sizing: border-box;
        }

        body {
            font-family: 'Poppins', sans-serif;
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5fbfb;
            color: var(--text-color);
            line-height: 1.6;
        }

        .form-container {
            perspective: 1000px;
        }

        form {
            background: var(--white);
            padding: 30px;
            border-radius: var(--border-radius);
            box-shadow: var(--box-shadow);
            transform-style: preserve-3d;
            transition: var(--transition);
            position: relative;
            overflow: hidden;
        }

        form::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 5px;
            background: linear-gradient(90deg, var(--primary-color), var(--accent-color));
        }

        form:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 25px rgba(0,0,0,0.15);
        }

        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 500;
            color: var(--primary-color);
            transition: var(--transition);
        }

        input, select {
            width: 100%;
            padding: 12px 15px;
            border: 2px solid #e0e0e0;
            border-radius: var(--border-radius);
            margin-bottom: 25px;
            font-size: 16px;
            font-family: 'Poppins', sans-serif;
            transition: var(--transition);
            background-color: var(--white);
        }

        input:focus, select:focus {
            border-color: var(--primary-color);
            outline: none;
            box-shadow: 0 0 0 3px rgba(31, 156, 157, 0.2);
        }

        input::placeholder {
            color: #aaa;
        }

        button {
            background-color: var(--primary-color);
            color: white;
            border: none;
            padding: 15px 30px;
            border-radius: var(--border-radius);
            cursor: pointer;
            font-size: 16px;
            font-weight: 600;
            width: 100%;
            transition: var(--transition);
            text-transform: uppercase;
            letter-spacing: 1px;
            position: relative;
            overflow: hidden;
        }

        button:hover {
            background-color: var(--primary-dark);
            transform: translateY(-2px);
        }

        button::after {
            content: '';
            position: absolute;
            top: 50%;
            left: 50%;
            width: 5px;
            height: 5px;
            background: rgba(255, 255, 255, 0.5);
            opacity: 0;
            border-radius: 100%;
            transform: scale(1, 1) translate(-50%);
            transform-origin: 50% 50%;
        }

        button:focus:not(:active)::after {
            animation: ripple 1s ease-out;
        }

        @keyframes ripple {
            0% {
                transform: scale(0, 0);
                opacity: 0.5;
            }
            100% {
                transform: scale(20, 20);
                opacity: 0;
            }
        }

        #loading {
            display: none;
            text-align: center;
            margin-top: 30px;
            color: var(--primary-color);
        }

        .loading-spinner {
            border: 4px solid var(--primary-light);
            border-top: 4px solid var(--primary-color);
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto 15px;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        #result {
            margin-top: 30px;
            padding: 25px;
            background: var(--white);
            border-radius: var(--border-radius);
            display: none;
            box-shadow: var(--box-shadow);
            border-left: 5px solid var(--primary-color);
            transform: translateY(20px);
            opacity: 0;
            animation: fadeInUp 0.5s forwards;
        }

        @keyframes fadeInUp {
            to {
                transform: translateY(0);
                opacity: 1;
            }
        }

        .error {
            background: #ffebee;
            color: #c62828;
            border-left: 5px solid #c62828;
        }

        .preference-section {
            margin-bottom: 30px;
            animation: fadeIn 0.5s ease-out;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .preference-title {
            color: var(--primary-color);
            margin-bottom: 15px;
            font-size: 18px;
            font-weight: 600;
            display: flex;
            align-items: center;
        }

        .preference-title::before {
            content: '';
            display: inline-block;
            width: 8px;
            height: 8px;
            background-color: var(--accent-color);
            border-radius: 50%;
            margin-right: 10px;
        }

        .preference-tabs {
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
            margin-bottom: 15px;
        }

        .preference-tab {
            padding: 10px 18px;
            background-color: var(--white);
            border: 2px solid var(--primary-color);
            border-radius: 30px;
            cursor: pointer;
            font-size: 14px;
            transition: var(--transition);
            color: var(--primary-color);
            font-weight: 500;
            position: relative;
            overflow: hidden;
        }

        .preference-tab:hover {
            background-color: var(--primary-light);
            transform: translateY(-2px);
        }

        .preference-tab.selected {
            background-color: var(--primary-color);
            color: var(--white);
            border-color: var(--primary-color);
            box-shadow: 0 4px 8px rgba(31, 156, 157, 0.2);
        }

        .preference-tab.selected::after {
            content: '✓';
            margin-left: 8px;
        }

        .preference-tab input {
            display: none;
        }

        .form-header {
            color: var(--primary-color);
            text-align: center;
            margin-bottom: 30px;
            position: relative;
            padding-bottom: 20px;
        }

        .form-header::after {
            content: '';
            position: absolute;
            bottom: 0;
            left: 50%;
            transform: translateX(-50%);
            width: 100px;
            height: 4px;
            background: linear-gradient(90deg, var(--primary-color), var(--accent-color));
            border-radius: 2px;
        }

        .form-header h2 {
            margin: 0;
            font-size: 32px;
            font-weight: 700;
            letter-spacing: -0.5px;
            animation: slideInDown 0.5s ease-out;
        }

        @keyframes slideInDown {
            from {
                transform: translateY(-20px);
                opacity: 0;
            }
            to {
                transform: translateY(0);
                opacity: 1;
            }
        }

        .form-header p {
            margin-top: 10px;
            color: var(--text-color);
            font-size: 16px;
            animation: fadeIn 0.8s ease-out;
        }

        .section-divider {
            height: 1px;
            background: linear-gradient(90deg, transparent, rgba(31, 156, 157, 0.3), transparent);
            margin: 30px 0;
        }

        .success-message {
            display: flex;
            align-items: center;
            margin-bottom: 20px;
        }

        .success-icon {
            width: 50px;
            height: 50px;
            background-color: var(--primary-light);
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            margin-right: 15px;
            flex-shrink: 0;
        }

        .success-icon svg {
            width: 24px;
            height: 24px;
            fill: var(--primary-color);
        }

        .view-link {
            display: inline-block;
            background-color: var(--primary-color);
            color: white;
            padding: 12px 25px;
            text-decoration: none;
            border-radius: var(--border-radius);
            margin-top: 15px;
            font-weight: 500;
            transition: var(--transition);
            box-shadow: 0 4px 8px rgba(31, 156, 157, 0.2);
        }

        .view-link:hover {
            background-color: var(--primary-dark);
            transform: translateY(-2px);
            box-shadow: 0 6px 12px rgba(31, 156, 157, 0.3);
        }

        .floating-icon {
            position: absolute;
            opacity: 0.1;
            z-index: -1;
        }

        .icon-plane {
            top: 50px;
            right: 50px;
            animation: float 6s ease-in-out infinite;
        }

        .icon-suitcase {
            bottom: 30px;
            left: 40px;
            animation: float 5s ease-in-out infinite 1s;
        }

        .icon-palm {
            top: 150px;
            left: 30px;
            animation: float 7s ease-in-out infinite 0.5s;
        }

        @keyframes float {
            0%, 100% {
                transform: translateY(0);
            }
            50% {
                transform: translateY(-15px);
            }
        }

        /* Responsive adjustments */
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }

            form {
                padding: 20px;
            }

            .form-header h2 {
                font-size: 26px;
            }

            .preference-tabs {
                gap: 8px;
            }

            .preference-tab {
                padding: 8px 14px;
                font-size: 13px;
            }
        }
    </style>
</head>
<body>
    <div class="form-container">
        <div class="form-header">
            <h2>Travel Itinerary Planner</h2>
            <p>Plan your perfect trip with personalized recommendations</p>
        </div>

        <form id="itineraryForm">
            <svg class="floating-icon icon-plane" width="80" height="80" viewBox="0 0 24 24" fill="var(--primary-color)">
                <path d="M22,16.21V18l-8-2.14V18l-2,1.07V16L2,18V3.79C2,2.8,2.93,2,4,2c0,0,6.5-1,8,0s8,0,8,0c1.07,0,2,.8,2,1.79V16.21z"/>
            </svg>

            <svg class="floating-icon icon-suitcase" width="60" height="60" viewBox="0 0 24 24" fill="var(--primary-color)">
                <path d="M19,6h-4V2H9v4H5v15h14V6z M11,4h2v2h-2V4z M5,8h14v3H5V8z M19,19H5v-5h14V19z"/>
            </svg>

            <svg class="floating-icon icon-palm" width="70" height="70" viewBox="0 0 24 24" fill="var(--primary-color)">
                <path d="M13,12v8h-2v-8H2.69c-1.31,0-1.89-1.57-1.06-2.43l8.34-8.34c0.39-0.39,1.02-0.39,1.41,0l8.34,8.34c0.83,0.86,0.25,2.43-1.06,2.43H13z"/>
            </svg>

            <div>
                <label for="location">Destination:</label>
                <input type="text" id="location" required placeholder="Where do you want to go?">
            </div>

            <div>
                <label for="no_of_days">Trip Duration (days):</label>
                <input type="number" id="no_of_days" min="1" max="30" required placeholder="How many days?">
            </div>

            <div>
                <label for="user_budget">Budget Level:</label>
                <select id="user_budget" required>
                    <option value="" disabled selected>Select your budget</option>
                    <option value="low">Low Budget</option>
                    <option value="medium">Medium Budget</option>
                    <option value="high">High Budget</option>
                    <option value="luxury">Luxury</option>
                </select>
            </div>

            <div class="section-divider"></div>

            <div class="preference-section" style="animation-delay: 0.1s">
                <div class="preference-title">Preferred Attraction Types</div>
                <div class="preference-tabs" id="attractionTypes">
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Wildlife"> Wildlife
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Sports"> Sports
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Cultural"> Cultural
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Leisure"> Leisure
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Religious"> Religious
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Historical"> Historical
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Natural"> Natural
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="attraction" value="Adventure"> Adventure
                    </label>
                </div>
            </div>

            <div class="preference-section" style="animation-delay: 0.2s">
                <div class="preference-title">Preferred Cuisines</div>
                <div class="preference-tabs" id="cuisineTypes">
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Sri Lankan"> Sri Lankan
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Vegetarian"> Vegetarian
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Middle Eastern"> Middle Eastern
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Seafood"> Seafood
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Thai"> Thai
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Italian"> Italian
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Indian"> Indian
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Latin American"> Latin American
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Japanese"> Japanese
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Chinese"> Chinese
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Caribbean"> Caribbean
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Turkish"> Turkish
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Mediterranean"> Mediterranean
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="South Asian"> South Asian
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="French"> French
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Fusion"> Fusion
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="International"> International
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="cuisine" value="Mexican"> Mexican
                    </label>
                </div>
            </div>

            <div class="preference-section" style="animation-delay: 0.3s">
                <div class="preference-title">Preferred Hotel Facilities</div>
                <div class="preference-tabs" id="facilityTypes">
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Free Wi-Fi"> Free Wi-Fi
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Air-conditioned"> Air-conditioned
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Free Parking"> Free Parking
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Pool (Outdoor/Indoor)"> Pool
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Restaurant"> Restaurant
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Kid-Friendly"> Kid-Friendly
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Room Service"> Room Service
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Beach Access"> Beach Access
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Spa"> Spa
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Laundry Service"> Laundry Service
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="facility" value="Mini Bar Drinks"> Mini Bar
                    </label>
                </div>
            </div>

            <div class="preference-section" style="animation-delay: 0.4s">
                <div class="preference-title">Activity Interests</div>
                <div class="preference-tabs" id="activityTypes">
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Adventure"> Adventure
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Cultural"> Cultural
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Culinary"> Culinary
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Wildlife & Nature"> Wildlife & Nature
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Wellness"> Wellness
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Historical"> Historical
                    </label>
                    <label class="preference-tab">
                        <input type="checkbox" name="activity" value="Leisure & Soft Adventure"> Leisure & Soft Adventure
                    </label>
                </div>
            </div>

            <div class="preference-section" style="animation-delay: 0.5s">
                <div class="preference-title">Guide Preferences</div>
                <div style="text-align: left;">
                    <label style="display: inline-flex; align-items: center; cursor: pointer;">
                        <input type="checkbox" id="need_guide" name="need_guide" onchange="toggleGuideLanguages()" style="margin-right: 10px; width: auto;">
                        <span>I would like a guide for my trip</span>
                    </label>
                </div>

                <div id="guideLanguagesSection" style="display: none; margin-top: 20px;">
                    <div class="preference-title">Preferred Guide Languages</div>
                    <div class="preference-tabs" id="guideLanguages">
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Sinhala"> Sinhala
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="English"> English
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Tamil"> Tamil
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="French"> French
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="German"> German
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Chinese"> Chinese
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Japanese"> Japanese
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Russian"> Russian
                        </label>
                        <label class="preference-tab">
                            <input type="checkbox" name="guide_language" value="Spanish"> Spanish
                        </label>
                    </div>
                </div>
            </div>

            <button type="button" onclick="generateItinerary()">
                <span>Generate Itinerary</span>
            </button>
        </form>

        <div id="loading">
            <div class="loading-spinner"></div>
            <p>Creating your personalized travel plan...</p>
        </div>

        <div id="result">
            <div class="success-message">
                <div class="success-icon">
                    <svg viewBox="0 0 24 24">
                        <path d="M9 16.17L4.83 12l-1.42 1.41L9 19 21 7l-1.41-1.41L9 16.17z"/>
                    </svg>
                </div>
                <div>
                    <h3>Your Dream Itinerary is Ready!</h3>
                    <p id="resultText"></p>
                </div>
            </div>
            <a id="viewLink" href="#" target="_blank" class="view-link">
                View Full Itinerary
            </a>
        </div>
    </div>

    <script>
    // Initialize tab selection behavior when DOM is loaded
    document.addEventListener('DOMContentLoaded', function() {
        // Add animation to form elements
        const sections = document.querySelectorAll('.preference-section');
        sections.forEach((section, index) => {
            section.style.animationDelay = `${0.1 + (index * 0.1)}s`;
        });

        // Tab selection functionality
        document.querySelectorAll('.preference-tab').forEach(tab => {
            const checkbox = tab.querySelector('input[type="checkbox"]');

            // Initialize selected state based on checkbox
            if (checkbox.checked) {
                tab.classList.add('selected');
            }

            tab.addEventListener('click', function(e) {
                // Prevent label from triggering checkbox twice
                if (e.target !== checkbox) {
                    checkbox.checked = !checkbox.checked;
                }

                // Update visual state
                if (checkbox.checked) {
                    tab.classList.add('selected');
                } else {
                    tab.classList.remove('selected');
                }

                // Add ripple effect
                const ripple = document.createElement('span');
                ripple.classList.add('ripple-effect');
                ripple.style.width = ripple.style.height = Math.max(tab.offsetWidth, tab.offsetHeight) + 'px';
                ripple.style.left = (e.offsetX - ripple.offsetWidth/2) + 'px';
                ripple.style.top = (e.offsetY - ripple.offsetHeight/2) + 'px';
                tab.appendChild(ripple);

                setTimeout(() => {
                    ripple.remove();
                }, 600);

                // Prevent default to avoid any form submission
                e.preventDefault();
            });
        });
    });

    function toggleGuideLanguages() {
        const needGuide = document.getElementById('need_guide').checked;
        const guideSection = document.getElementById('guideLanguagesSection');

        if (needGuide) {
            guideSection.style.display = 'block';
            setTimeout(() => {
                guideSection.style.opacity = '1';
                guideSection.style.transform = 'translateY(0)';
            }, 10);
        } else {
            guideSection.style.opacity = '0';
            guideSection.style.transform = 'translateY(-10px)';
            setTimeout(() => {
                guideSection.style.display = 'none';
            }, 300);
        }
    }

    function generateItinerary() {
        // Show loading indicator
        document.getElementById('loading').style.display = 'block';
        document.getElementById('result').style.display = 'none';

        // Get form values
        const formData = {
            location: document.getElementById('location').value,
            no_of_days: document.getElementById('no_of_days').value,
            user_budget: document.getElementById('user_budget').value,
            user_attraction_visit_preferences: Array.from(document.querySelectorAll('input[name="attraction"]:checked')).map(el => el.value),
            user_cuisine_preferences: Array.from(document.querySelectorAll('input[name="cuisine"]:checked')).map(el => el.value),
            user_facilities: Array.from(document.querySelectorAll('input[name="facility"]:checked')).map(el => el.value),
            user_activity_interests: Array.from(document.querySelectorAll('input[name="activity"]:checked')).map(el => el.value),
            need_guide: document.getElementById('need_guide').checked,
            user_guide_languages: Array.from(document.querySelectorAll('input[name="guide_language"]:checked')).map(el => el.value)
        };

        // Validate required fields
        if (!formData.location || !formData.no_of_days || !formData.user_budget) {
            document.getElementById('loading').style.display = 'none';
            showError('Please fill in all required fields.');
            return;
        }

        // Validate guide languages if guide is requested
        if (formData.need_guide && formData.user_guide_languages.length === 0) {
            document.getElementById('loading').style.display = 'none';
            showError('Please select at least one guide language.');
            return;
        }

        // Add button click animation
        const button = document.querySelector('button');
        button.classList.add('clicked');
        setTimeout(() => {
            button.classList.remove('clicked');
        }, 300);

        // Send data to server
        fetch('/generate', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify(formData)
        })
        .then(response => {
            if (!response.ok) {
                throw new Error('Network response was not ok');
            }
            return response.json();
        })
        .then(data => {
            document.getElementById('loading').style.display = 'none';

            if (data.success) {
                document.getElementById('resultText').textContent =
                    `Your ${formData.no_of_days}-day trip to ${formData.location} is ready! We've customized it based on your preferences.`;
                document.getElementById('viewLink').href = data.itinerary_url;
                document.getElementById('result').style.display = 'block';
                document.getElementById('result').className = '';

                // Redirect to the itinerary page after a short delay
                setTimeout(() => {
                    window.location.href = data.itinerary_url;
                }, 1500);
            } else {
                showError(data.error || 'We encountered an issue generating your itinerary. Please try again later.');
            }
        })
        .catch(error => {
            document.getElementById('loading').style.display = 'none';
            showError('Error: ' + error.message);
        });
    }

    function showError(message) {
        document.getElementById('resultText').textContent = message;
        document.getElementById('result').style.display = 'block';
        document.getElementById('result').className = 'error';
    }
    </script>
</body>
</html>
    """

In [ ]:
#Cell 06
@app.route('/generate', methods=['POST'])
def generate_itinerary():
    try:
        form_data = request.json

        # Generate itinerary using the global model and data
        itinerary = generate_itinerary_plan(
            model,
            global_data['activity_data'],
            global_data['hotels_data'],
            global_data['restaurants_data'],
            global_data['activity_data_new'],
            global_data['transport_data'],
            global_data['guides_data'],
            form_data['user_budget'],
            form_data['location'],
            int(form_data['no_of_days']),
            form_data['user_attraction_visit_preferences'],
            form_data['user_facilities'],
            form_data['user_cuisine_preferences'],
            form_data['user_activity_interests'],
            form_data.get('need_guide', False),
            form_data.get('user_guide_languages', [])
        )

        # Generate HTML
        final_html = generate_html_itinerary(
            itinerary,
            form_data['location'],
            int(form_data['no_of_days']),
            form_data['user_budget'],
            form_data['user_attraction_visit_preferences'],
            form_data['user_cuisine_preferences']
        )

        # Save HTML file in the static folder
        if not os.path.exists('static'):
            os.makedirs('static')

        html_file_path = f"static/itinerary_{form_data['location'].lower().replace(' ', '_')}_{form_data['no_of_days']}days.html"
        with open(html_file_path, 'w') as file:
            file.write(final_html)

        return jsonify({
            'success': True,
            'itinerary_url': f'/{html_file_path}'
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        })

In [ ]:
#Cell 07
@app.route('/static/<path:path>')
def serve_static(path):
    return send_from_directory('static', path)


In [ ]:
#Cell 08
# Data loading and processing functions
def load_data():
    # Load data from GitHub
    activity_data, hotels_data, restaurants_data, activity_data_new, transport_data, guides_data = load_github_data()

    # Preprocess the data
    hotels_data, restaurants_data, transport_data = preprocess_data(hotels_data, restaurants_data, transport_data)

    return activity_data, hotels_data, restaurants_data, activity_data_new, transport_data, guides_data

In [ ]:
#cell 09
def preprocess_data(hotels_data, restaurants_data, transport_data):
    # Process hotels data
    for hotel in hotels_data:
        hotel["Price_per_Night_(LKR)"] = float(hotel.get("Price_per_Night_(LKR)", 0))
        amenities = hotel.get("Property_amenities", "")
        if isinstance(amenities, str):
            hotel["Property_amenities"] = [amenity.strip() for amenity in amenities.split(",")]
        hotel["Budget_category"] = hotel.get("Budget_category", "").strip().lower()

    # Process restaurants data
    for restaurant in restaurants_data:
        price = restaurant.get("Price per Person", "0")
        restaurant["Price per Person"] = float(price) if price else 0.0
        restaurant["Budget Range"] = restaurant.get("Budget Range", "").strip().lower()
        restaurant["Meal Type"] = restaurant.get("Meal Type", "").strip().lower()
        cuisine = restaurant.get("Cuisine Type", "")
        if isinstance(cuisine, str):
            restaurant["Cuisine Type"] = [c.strip() for c in cuisine.split(",")]
        else:
            restaurant["Cuisine Type"] = []

   # Process transport data
    for transport in transport_data:
        # Convert price to float
        price = transport.get("Price Per Day (LKR)", "0")
        transport["Price Per Day (LKR)"] = float(price) if price else 0.0

        # Ensure category is lowercase
        transport["Category"] = transport.get("Category", "").strip().lower()

        # Ensure location is properly formatted
        transport["Location"] = transport.get("Location", "").strip()

        # Ensure all fields are present
        transport["Transport Method"] = transport.get("Transport Method", "N/A")
        transport["Vehicle Details"] = transport.get("Vehicle Details", "N/A")
        transport["Contact Number"] = transport.get("Contact Number", "N/A")
        transport["Driver_Name"] = transport.get("Driver_Name", "N/A")
        transport["Driver License Number"] = transport.get("Driver License Number", "N/A")
        transport["Additional Services"] = transport.get("Additional Services", "N/A")
    return hotels_data, restaurants_data, transport_data

# Itinerary generation functions
def retrieve_attractions(model, activity_data, query, budget_range, preferred_categories, top_k=5):
    # Normalize inputs
    budget_range = budget_range.lower()
    preferred_categories = [cat.strip().lower() for cat in preferred_categories]

    filtered_attractions = []
    for activity in activity_data:
        # Check budget match
        budget_match = activity.get("Budget_range", "").lower() == budget_range

        # Check category match with flexible comparison
        category_match = False
        activity_category = activity.get("Category", "").lower()

        # Check if any preferred category is contained in the activity category or vice versa
        for pref_cat in preferred_categories:
            if pref_cat in activity_category or activity_category in pref_cat:
                category_match = True
                break

        if budget_match and category_match:
            filtered_attractions.append(activity)

    if not filtered_attractions:
        return []

    # Rest of your existing similarity calculation code
    filtered_descriptions = [
        f"{activity.get('Name', '')}: {activity.get('Location', '')}. Duration: {activity.get('Average_Visit_Duration', '')}"
        for activity in filtered_attractions
    ]

    filtered_embeddings = model.encode(filtered_descriptions)
    query_embedding = model.encode([query])[0].tolist()
    similarity_scores = np.dot(filtered_embeddings, query_embedding)
    ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]

    return [filtered_attractions[i] for i in ranked_indices]

def retrieve_hotels(model, hotels_data, query, budget_range, top_k=5):
    filtered_hotels = [hotel for hotel in hotels_data if hotel.get("Budget_category", "") == budget_range]
    filtered_descriptions = [
        f"{hotel.get('Hotel_name', '')}: {hotel.get('Location', '')}. {', '.join(hotel.get('Property_amenities', []))}"
        for hotel in filtered_hotels
    ]
    if not filtered_descriptions:
        return []
    filtered_embeddings = model.encode(filtered_descriptions)
    query_embedding = model.encode([query])[0].tolist()
    similarity_scores = np.dot(filtered_embeddings, query_embedding)
    ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]
    return [filtered_hotels[i] for i in ranked_indices] if filtered_hotels else []

def retrieve_restaurants(model, restaurants_data, query, budget_range, meal_type, cuisine_preferences, location, top_k=5):
    filtered_restaurants = [
        restaurant for restaurant in restaurants_data
        if (restaurant.get("Budget Range", "") == budget_range and
            restaurant.get("Meal Type", "") == meal_type and
            location.lower() in restaurant.get("Location", "").lower() and
            any(cuisine.lower() in [c.lower() for c in restaurant.get("Cuisine Type", [])]
                for cuisine in cuisine_preferences)
        )
    ]
    if not filtered_restaurants:
        return []
    filtered_descriptions = [
        f"{restaurant.get('Restaurant Name', '')}: {restaurant.get('Location', '')}. {', '.join(restaurant.get('Cuisine Type', []))}"
        for restaurant in filtered_restaurants
    ]
    filtered_embeddings = model.encode(filtered_descriptions)
    query_embedding = model.encode([query])[0].tolist()
    similarity_scores = np.dot(filtered_embeddings, query_embedding)
    ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]
    return [filtered_restaurants[i] for i in ranked_indices] if filtered_restaurants else []

def retrieve_transport(transport_data, budget_range, location, num_providers):
    filtered_transport = [
        transport for transport in transport_data
        if transport.get("Category", "") == budget_range.lower() and
           location.lower() in transport.get("Location", "").lower()
    ]
    return random.sample(filtered_transport, min(num_providers, len(filtered_transport))) if filtered_transport else []


def filter_activities(activity_data, budget_range, interests, location, used_activity_ids=None):
    if used_activity_ids is None:
        used_activity_ids = set()

    return [
        activity for activity in activity_data
        if (activity.get('Budget_range', '').lower() == budget_range.lower() and
            activity.get('Interest', '').lower() in [interest.lower() for interest in interests] and
            location.lower() in activity.get('Location', '').lower() and
            activity.get('Activity name') not in used_activity_ids)
    ]

def retrieve_guides(guides_data, budget_range, languages, num_days):
    # Filter guides by budget and language
    filtered_guides = [
        guide for guide in guides_data
        if (guide.get("budgetRange", "").lower() == budget_range.lower() and
            any(lang in guide.get("languagesSpoken", []) for lang in languages))
    ]

    if not filtered_guides:
        return []

    # Determine how many guides we need (one per 5 days)
    num_guides_needed = (num_days + 4) // 5

    # If we don't have enough unique guides, we'll reuse some
    if len(filtered_guides) >= num_guides_needed:
        return random.sample(filtered_guides, num_guides_needed)
    else:
        # If not enough unique guides, fill with random selections from available
        result = filtered_guides.copy()
        while len(result) < num_guides_needed:
            result.append(random.choice(filtered_guides))
        return result


In [ ]:
 #Cell 10
 def generate_itinerary_plan(model, activity_data, hotels_data, restaurants_data,
                          activity_data_new, transport_data, guides_data,
                          user_budget, location, no_of_days,
                          user_attraction_visit_preferences, user_facilities,
                          user_cuisine_preferences, user_activity_interests,
                          need_guide=False, guide_languages=None):

    itinerary = {}
    total_cost = 0
    used_activity_ids = set()
    used_attraction_ids = set()
    used_restaurant_ids = set()  # Track used restaurants
    used_hotel_names = set()    # Track used hotels
    used_transport_ids = set()  # Track used transport providers

    # Retrieve transport providers
    num_transport_providers = max(1, (no_of_days + 2) // 3)
    transport_providers = [
        t for t in transport_data
        if (t.get("Category", "").lower() == user_budget.lower() and
            location.lower() in t.get("Location", "").lower() and
            t.get("Provider Name") not in used_transport_ids)
    ]

    # If we don't have enough unique transport providers, allow duplicates
    if len(transport_providers) < num_transport_providers:
        transport_providers = [
            t for t in transport_data
            if (t.get("Category", "").lower() == user_budget.lower() and
                location.lower() in t.get("Location", "").lower())
        ]

    # Retrieve accommodations
    num_accommodations = max(1, (no_of_days + 2) // 3)
    hotel_query = f"Find a {user_budget} hotel in {location} with {', '.join(user_facilities)}."
    recommended_hotels = [
        h for h in hotels_data
        if (h.get("Budget_category", "").lower() == user_budget.lower() and
           h.get("Hotel_name") not in used_hotel_names)
    ]

    # If we don't have enough unique hotels, allow duplicates
    if len(recommended_hotels) < num_accommodations:
        recommended_hotels = [
            h for h in hotels_data
            if h.get("Budget_category", "").lower() == user_budget.lower()
        ]

    # Retrieve guides if needed
    selected_guides = []
    if need_guide and guide_languages:
        guide_candidates = [
            g for g in guides_data
            if (g.get("budgetRange", "").lower() == user_budget.lower() and
               any(lang in g.get("languagesSpoken", []) for lang in guide_languages))
        ]

        num_guides_needed = max(1, (no_of_days + 4) // 5)
        if len(guide_candidates) >= num_guides_needed:
            selected_guides = random.sample(guide_candidates, num_guides_needed)
        elif guide_candidates:
            selected_guides = guide_candidates.copy()
            while len(selected_guides) < num_guides_needed:
                selected_guides.append(random.choice(guide_candidates))

    # Generate daily plan
    for day in range(1, no_of_days + 1):
        hotel_index = (day - 1) % len(recommended_hotels)
        transport_index = (day - 1) % len(transport_providers) if transport_providers else 0

        selected_hotel = recommended_hotels[hotel_index]
        used_hotel_names.add(selected_hotel.get("Hotel_name"))

        selected_transport = transport_providers[transport_index] if transport_providers else None
        if selected_transport:
            used_transport_ids.add(selected_transport.get("Provider Name"))

        # Assign guide (one guide per 5 days)
        selected_guide = None
        if need_guide and selected_guides:
            guide_index = (day - 1) // 5
            selected_guide = selected_guides[guide_index % len(selected_guides)]

        # Attraction selection with multiple fallback levels
        preferred_category = user_attraction_visit_preferences[(day-1) % len(user_attraction_visit_preferences)]
        selected_attraction = None

        fallback_levels = [
            # Level 1: Exact match for category and budget in location
            lambda: [
                attr for attr in activity_data
                if (attr.get("Budget_range", "").lower() == user_budget.lower() and
                    attr.get("Category", "").lower() == preferred_category.lower() and
                    attr.get("Attraction_ID") not in used_attraction_ids and
                    location.lower() in attr.get("Location", "").lower())
            ],

            # Level 2: Same category but any budget in location
            lambda: [
                attr for attr in activity_data
                if (attr.get("Category", "").lower() == preferred_category.lower() and
                    attr.get("Attraction_ID") not in used_attraction_ids and
                    location.lower() in attr.get("Location", "").lower())
            ],

            # Level 3: Any attraction in location matching any preferred category
            lambda: [
                attr for attr in activity_data
                if (attr.get("Category", "").lower() in [c.lower() for c in user_attraction_visit_preferences] and
                    attr.get("Attraction_ID") not in used_attraction_ids and
                    location.lower() in attr.get("Location", "").lower())
            ],

            # Level 4: Any attraction in location
            lambda: [
                attr for attr in activity_data
                if (attr.get("Attraction_ID") not in used_attraction_ids and
                    location.lower() in attr.get("Location", "").lower())
            ],

            # Level 5: Any attraction matching category (regardless of location)
            lambda: [
                attr for attr in activity_data
                if (attr.get("Category", "").lower() == preferred_category.lower() and
                    attr.get("Attraction_ID") not in used_attraction_ids)
            ]
        ]

        for fallback in fallback_levels:
            filtered_attractions = fallback()
            if filtered_attractions:
                selected_attraction = random.choice(filtered_attractions)
                used_attraction_ids.add(selected_attraction.get("Attraction_ID"))
                break

        if not selected_attraction:
            # Final fallback - create a meaningful suggestion
            popular_activities = {
                "Historical": f"Explore {location}'s historical landmarks and architecture",
                "Cultural": f"Visit local cultural centers or museums in {location}",
                "Adventure": f"Try outdoor activities around {location}",
                "Nature": f"Enjoy nature walks in {location}'s scenic areas",
                "Religious": f"Visit spiritual sites in {location}",
                "Leisure": f"Stroll through {location}'s markets and shopping areas",
                "Wildlife": f"Explore wildlife areas near {location}",
                "Sports": f"Enjoy sports activities in {location}"
            }
            description = popular_activities.get(preferred_category,
                             f"Explore {location}'s local attractions and scenery")

            selected_attraction = {
                "Name": f"{preferred_category} Experience in {location}",
                "Location": location,
                "Average_Visit_Duration": "2-3 hours",
                "Category": preferred_category,
                "Entry_Fee": "0",
                "Description": description,
                "Budget_range": user_budget,
                "Attraction_ID": f"fallback_{day}"
            }

        # Activity selection with improved filtering
        selected_activity = None
        filtered_activities = [
            act for act in activity_data_new
            if (act.get('Budget_range', '').lower() == user_budget.lower() and
                act.get('Interest', '').lower() in [interest.lower() for interest in user_activity_interests] and
                location.lower() in act.get('Location', '').lower() and
                act.get('Activity name') not in used_activity_ids)
        ]

        # If no exact matches, relax some criteria
        if not filtered_activities:
            filtered_activities = [
                act for act in activity_data_new
                if (act.get('Budget_range', '').lower() == user_budget.lower() and
                    location.lower() in act.get('Location', '').lower() and
                    act.get('Activity name') not in used_activity_ids)
            ]

        # If still no matches, just get any activity in location
        if not filtered_activities:
            filtered_activities = [
                act for act in activity_data_new
                if location.lower() in act.get('Location', '').lower() and
                   act.get('Activity name') not in used_activity_ids
            ]

        if filtered_activities:
            selected_activity = random.choice(filtered_activities)
            used_activity_ids.add(selected_activity.get('Activity name'))
        else:
            # Create a meaningful fallback activity
            selected_activity = {
                "Activity name": f"Local Experience in {location}",
                "Interest": random.choice(user_activity_interests) if user_activity_interests else "General",
                "Location": location,
                "Price": "0",
                "Duration": "2-3 hours",
                "Description": f"Enjoy a local experience in {location}",
                "Budget_range": user_budget,
                "Activity name": f"fallback_{day}"  # Unique identifier for fallback activities
            }

        daily_plan = {
            "hotel": selected_hotel,
            "attraction": selected_attraction,
            "activity": selected_activity,
            "meals": {},
            "transport": selected_transport,
            "guide": selected_guide
        }

        # Assign meals with improved restaurant selection
        for meal_type in ["breakfast", "lunch", "dinner"]:
            # First try to find restaurants that match all criteria and haven't been used
            matching_restaurants = [
                r for r in restaurants_data
                if (r.get("Budget Range", "").lower() == user_budget.lower() and
                   r.get("Meal Type", "").lower() == meal_type.lower() and
                   location.lower() in r.get("Location", "").lower() and
                   any(cuisine.lower() in [c.lower() for c in r.get("Cuisine Type", [])]
                       for cuisine in user_cuisine_preferences) and
                   r.get("Restaurant Name") not in used_restaurant_ids)
            ]

            # If no unused restaurants found, consider all matching restaurants
            if not matching_restaurants:
                matching_restaurants = [
                    r for r in restaurants_data
                    if (r.get("Budget Range", "").lower() == user_budget.lower() and
                       r.get("Meal Type", "").lower() == meal_type.lower() and
                       location.lower() in r.get("Location", "").lower() and
                       any(cuisine.lower() in [c.lower() for c in r.get("Cuisine Type", [])]
                           for cuisine in user_cuisine_preferences))
                ]

            # Select the most relevant restaurant using similarity scoring
            if matching_restaurants:
                # Encode descriptions for similarity comparison
                descriptions = [
                    f"{r.get('Restaurant Name', '')}: {r.get('Location', '')}. {', '.join(r.get('Cuisine Type', []))}"
                    for r in matching_restaurants
                ]
                embeddings = model.encode(descriptions)
                query_embedding = model.encode([f"{meal_type} restaurant in {location}"])[0].tolist()
                similarity_scores = np.dot(embeddings, query_embedding)
                best_index = np.argmax(similarity_scores)
                selected_restaurant = matching_restaurants[best_index]

                # Add to used restaurants if it's not a duplicate fallback
                if len(matching_restaurants) > 1:  # Only track if we had multiple options
                    used_restaurant_ids.add(selected_restaurant.get("Restaurant Name"))
            else:
                # Fallback if no restaurants found
                selected_restaurant = {
                    "Restaurant Name": f"Local {meal_type} spot",
                    "Price per Person": 0,
                    "Location": location,
                    "Cuisine Type": user_cuisine_preferences,
                    "Budget Range": user_budget,
                    "Meal Type": meal_type
                }

            daily_plan["meals"][meal_type] = selected_restaurant
            total_cost += selected_restaurant.get("Price per Person", 0)

        itinerary[f"Day {day}"] = daily_plan

        # Calculate costs
        total_cost += selected_hotel.get("Price_per_Night_(LKR)", 0)

        if selected_transport:
            total_cost += selected_transport.get("Price Per Day (LKR)", 0)

        if selected_activity:
            total_cost += float(selected_activity.get('Price', 0))

        if selected_attraction and selected_attraction.get("Entry_Fee", "0").isdigit():
            total_cost += float(selected_attraction.get("Entry_Fee", "0"))

        if selected_guide:
            total_cost += float(selected_guide.get("pricePerDay", 0))

    itinerary["total_cost"] = total_cost
    return itinerary

In [ ]:
#Cell 11
def generate_html_itinerary(itinerary, location, no_of_days, user_budget,
                          user_attraction_visit_preferences, user_cuisine_preferences):
    days_html = ""
    for day_key, day_data in [(k, v) for k, v in itinerary.items() if k.startswith("Day")]:
        days_html += create_day_html(day_key, day_data)

    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Travel Itinerary</title>
        <style>
            :root {
                --primary-color: #1f9c9d;
                --secondary-color: #FFAA1D;
                --dark-color: #2c3e50;
                --light-color: #f8f9fa;
                --white: #ffffff;
                --text-color: #333;
                --border-radius: 8px;
                --box-shadow: 0 4px 12px rgba(0,0,0,0.1);
                --transition: all 0.3s ease;
            }

            body {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                line-height: 1.6;
                color: var(--text-color);
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
                background-color: var(--white);
            }

            header {
                background: linear-gradient(135deg, var(--primary-color), var(--dark-color));
                color: var(--white);
                padding: 30px 20px;
                border-radius: var(--border-radius);
                margin-bottom: 30px;
                text-align: center;
                box-shadow: var(--box-shadow);
                transition: var(--transition);
            }

            header:hover {
                transform: translateY(-2px);
                box-shadow: 0 6px 16px rgba(0,0,0,0.15);
            }

            h1 {
                margin: 0;
                font-size: 2.5em;
                text-shadow: 1px 1px 3px rgba(0,0,0,0.2);
            }

            .trip-summary {
                background-color: var(--light-color);
                padding: 20px;
                border-radius: var(--border-radius);
                margin-bottom: 30px;
                border-left: 5px solid var(--primary-color);
                transition: var(--transition);
            }

            .trip-summary:hover {
                transform: translateX(5px);
            }

            .day-card {
                background-color: var(--white);
                border-radius: var(--border-radius);
                box-shadow: var(--box-shadow);
                margin-bottom: 40px;
                overflow: hidden;
                transition: var(--transition);
            }

            .day-card:hover {
                transform: translateY(-5px);
                box-shadow: 0 8px 20px rgba(0,0,0,0.15);
            }

            .day-header {
                background-color: #1f9c9d; /* Solid teal color */
    color: var(--white);
    padding: 18px 25px;
    font-size: 1.5em;
    display: flex;
    justify-content: space-between;
    align-items: center;
    border-radius: var(--border-radius) var(--border-radius) 0 0; /* Rounded top corners only */
    transition: var(--transition); /* Smooth hover effect */
}

            .day-header span {
                font-weight: bold;
                text-shadow: 1px 1px 2px rgba(0,0,0,0.2);
            }

            .section {
                padding: 25px;
                border-bottom: 1px solid rgba(0,0,0,0.05);
                transition: var(--transition);
            }

            .section:hover {
                background-color: rgba(31, 156, 157, 0.03);
            }

            .section:last-child {
                border-bottom: none;
            }

            .section-header {
                color: var(--primary-color);
                margin-top: 0;
                font-size: 1.3em;
                display: flex;
                align-items: center;
                margin-bottom: 15px;
                padding-bottom: 10px;
                border-bottom: 2px solid var(--secondary-color);
            }

            .section-header i {
                margin-right: 12px;
                color: var(--secondary-color);
                font-size: 1.2em;
            }

            .hotel-info,
            .attraction-info,
            .activity-info,
            .meal-info,
            .transport-info {
                display: flex;
                flex-wrap: wrap;
                gap: 25px;
            }

            .info-card {
                flex: 1;
                min-width: 250px;
                background-color: var(--light-color);
                padding: 20px;
                border-radius: var(--border-radius);
                box-shadow: 0 2px 8px rgba(0,0,0,0.05);
                transition: var(--transition);
                border-top: 3px solid var(--primary-color);
            }

            .info-card:hover {
                transform: translateY(-3px);
                box-shadow: 0 6px 15px rgba(0,0,0,0.1);
                border-top-color: var(--secondary-color);
            }

            .info-card h3 {
                margin-top: 0;
                color: var(--primary-color);
                padding-bottom: 8px;
                border-bottom: 1px dashed var(--secondary-color);
            }

            .price {
                font-weight: bold;
                color: var(--secondary-color);
                font-size: 1.1em;
            }

            .facilities, .cuisine {
                font-style: italic;
                color: #6c757d;
                background-color: rgba(255, 170, 29, 0.1);
                padding: 5px 10px;
                border-radius: 4px;
                display: inline-block;
                margin-top: 5px;
            }

            .total-cost {
                background: linear-gradient(to right, var(--dark-color), var(--primary-color));
                color: var(--white);
                padding: 20px;
                border-radius: var(--border-radius);
                text-align: center;
                font-size: 1.3em;
                margin-top: 40px;
                box-shadow: var(--box-shadow);
                transition: var(--transition);
            }

            .total-cost:hover {
                transform: scale(1.02);
            }

            .total-cost h2 {
                margin: 0;
                text-shadow: 1px 1px 3px rgba(0,0,0,0.3);
            }

            /* Animation for cards */
            @keyframes fadeIn {
                from { opacity: 0; transform: translateY(20px); }
                to { opacity: 1; transform: translateY(0); }
            }

            .day-card {
                animation: fadeIn 0.5s ease forwards;
            }

            /* Responsive design */
            @media (max-width: 768px) {
                .hotel-info,
                .attraction-info,
                .activity-info,
                .meal-info,
                .transport-info {
                    flex-direction: column;
                    gap: 15px;
                }

                .info-card {
                    min-width: 100%;
                }

                header {
                    padding: 20px 15px;
                }

                h1 {
                    font-size: 2em;
                }
            }

            /* Tooltip style for additional info */
            [data-tooltip] {
                position: relative;
                cursor: pointer;
            }

            [data-tooltip]:hover::after {
                content: attr(data-tooltip);
                position: absolute;
                bottom: 100%;
                left: 50%;
                transform: translateX(-50%);
                background: var(--dark-color);
                color: var(--white);
                padding: 5px 10px;
                border-radius: 4px;
                font-size: 0.8em;
                white-space: nowrap;
                z-index: 10;
            }
        </style>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
    </head>
    <body>
        <header>
            <h1>Your Travel Itinerary</h1>
            <p>{location} • {no_of_days} Days • {user_budget} Budget</p>
        </header>

        <div class="trip-summary">
            <h2>Trip Overview</h2>
            <p>This {no_of_days}-day trip to {location} is tailored to your preferences with a {user_budget} budget.
            You'll experience {attraction_prefs} attractions and enjoy {cuisine_prefs} cuisine throughout your stay.</p>
        </div>

        {days_html}

        <div class="total-cost">
            <h2>Estimated Total Cost: LKR {total_cost}</h2>
        </div>
    </body>
    </html>
    """

    return html_template.replace("{location}", location)\
                      .replace("{no_of_days}", str(no_of_days))\
                      .replace("{user_budget}", user_budget.capitalize())\
                      .replace("{attraction_prefs}", ", ".join(user_attraction_visit_preferences))\
                      .replace("{cuisine_prefs}", ", ".join(user_cuisine_preferences))\
                      .replace("{days_html}", days_html)\
                      .replace("{total_cost}", f"{float(itinerary.get('total_cost', 0)):,.2f}")


def create_day_html(day, day_data):
    sections = []

    # Hotel section
    if day_data.get("hotel"):
        hotel = day_data["hotel"]
        sections.append(f"""
        <div class="section">
            <h3 class="section-header"><i class="fas fa-hotel"></i> Accommodation</h3>
            <div class="hotel-info">
                <div class="info-card">
                    <h3>{hotel.get('Hotel_name', 'No hotel specified')}</h3>
                    <p><strong>Location:</strong> {hotel.get('Location', 'N/A')}</p>
                    <p><strong>Price per night:</strong> <span class="price">LKR {hotel.get('Price_per_Night_(LKR)', 0):,.2f}</span></p>
                    <p class="facilities"><strong>Facilities:</strong> {', '.join(hotel.get('Property_amenities', [])) if hotel.get('Property_amenities') else 'N/A'}</p>
                </div>
            </div>
        </div>
        """)

    # Attraction section
    if day_data.get("attraction"):
        attraction = day_data["attraction"]
        sections.append(f"""
        <div class="section">
            <h3 class="section-header"><i class="fas fa-map-marked-alt"></i> Attraction</h3>
            <div class="attraction-info">
                <div class="info-card">
                    <h3>{attraction.get('Name', 'No attraction specified')}</h3>
                    <p><strong>Category:</strong> {attraction.get('Category', 'N/A')}</p>
                    <p><strong>Location:</strong> {attraction.get('Location', 'N/A')}</p>
                    <p><strong>Opening Hours:</strong> {attraction.get('Opening_Hours', 'N/A')}</p>
                    <p><strong>Best time to visit:</strong> {attraction.get('Best_Time_to_Visit','N/A')}</p>
                    <p><strong>Facilities:</strong> {attraction.get('Opening_Hours', 'N/A')}</p>
                    <p><strong>Description:</strong> {attraction.get('Facilities_Available', 'N/A')}</p>
                </div>
            </div>
        </div>
        """)

    # Activity section
    if day_data.get("activity"):
        activity = day_data["activity"]
        sections.append(f"""
        <div class="section">
            <h3 class="section-header"><i class="fas fa-hiking"></i> Activity</h3>
            <div class="activity-info">
                <div class="info-card">
                    <h3>{activity.get('Activity name', 'No activity specified')}</h3>
                    <p><strong>Provider:</strong> {activity.get('Activity _Provider_Name', 'N/A')}</p>
                    <p><strong>Category:</strong> {activity.get('Interest', 'N/A')}</p>
                    <p><strong>Price:</strong> <span class="price">LKR {float(activity.get('Price', 0)):,.2f}</span></p>
                    <p><strong>Duration:</strong> {activity.get('Duration', 'N/A')}</p>
                    <p><strong>Contact:</strong> {"+94 758625849"}</p>
                </div>
            </div>
        </div>
        """)

    # Meals section
    if day_data.get("meals"):
        meals = day_data["meals"]
        meals_html = """
        <div class="section">
            <h3 class="section-header"><i class="fas fa-utensils"></i> Meals</h3>
            <div class="meal-info">
        """

        for meal_type, meal in meals.items():
            if meal and isinstance(meal, dict):
                meals_html += f"""
                <div class="info-card">
                    <h3>{meal_type.capitalize()}</h3>
                    <p><strong>Restaurant:</strong> {meal.get('Restaurant Name', 'No restaurant specified')}</p>
                    <p><strong>Cuisine:</strong> <span class="cuisine">{', '.join(meal.get('Cuisine Type', [])) if meal.get('Cuisine Type') else 'N/A'}</span></p>
                    <p><strong>Price per person:</strong> <span class="price">LKR {float(meal.get('Price per Person', 0)):,.2f}</span></p>
                    <p><strong>Opening Hours:</strong> {meal.get('Opening Hours', '24/7')}</p>
                    <p><strong>Contact:</strong> {meal.get('Contact Number', 'N/A')}</p>
                    <p><strong>Location:</strong> {meal.get('Location', 'N/A')}</p>
                </div>
                """

        meals_html += """
            </div>
        </div>
        """
        sections.append(meals_html)

          # Guide section
    if day_data.get("guide"):
        guide = day_data["guide"]
        sections.append(f"""
        <div class="section">
            <h3 class="section-header"><i class="fas fa-user-tie"></i> Guide</h3>
            <div class="guide-info">
                <div class="info-card">
                    <h3>{guide.get('fullName', 'No guide specified')}</h3>
                    <p><strong>Languages:</strong> {', '.join(guide.get('languagesSpoken', []))}</p>
                    <p><strong>Price per day:</strong> <span class="price">LKR {float(guide.get('pricePerDay', 0)):,.2f}</span></p>
                    <p><strong>Rating:</strong> {guide.get('ratings', 'N/A')}/5</p>
                    <p><strong>Contact:</strong>{guide.get('contactNumber', 'N/A')}</p>
                </div>
            </div>
        </div>
        """)

    # Transport section
    if day_data.get("transport"):
        transport = day_data["transport"]
        sections.append(f"""
        <div class="section">
            <h3 class="section-header"><i class="fas fa-car"></i> Transport</h3>
            <div class="transport-info">
                <div class="info-card">
                    <h3>{transport.get('Provider Name', 'No transport specified')}</h3>
                    <p><strong>Vehicle Type:</strong> {transport.get('Transport Method', 'N/A')}</p>
                    <p><strong>Vehicle Details:</strong> {transport.get('Vehicle Details', 'N/A')}</p>
                    <p><strong>Price per day:</strong> <span class="price">LKR {float(transport.get('Price Per Day (LKR)', 0)):,.2f}</span></p>
                    <p><strong>Driver:</strong> {transport.get('Driver_Name', 'N/A')}</p>
                    <p><strong>Contact:</strong> <a href="tel:{transport.get('Contact Number', '')}">{transport.get('Contact Number', 'N/A')}</a></p>
                    <p><strong>License:</strong> {transport.get('Driver License Number', 'N/A')}</p>
                    <p><strong>Services:</strong> {transport.get('Additional Services', 'N/A')}</p>
                </div>
            </div>
        </div>
        """)

    return f"""
    <div class="day-card">
        <div class="day-header">
            <span>{day}</span>
        </div>
        {"".join(sections)}
    </div>
    """

In [ ]:
#Cell 12
# Initialize the system
def initialize_system():
    global model, pc, global_data

    # Initialize models and services
    model = SentenceTransformer('all-MiniLM-L6-v2')
    pc = Pinecone(api_key="your-pinecone-api-key")

    # Load and preprocess data
    activity_data, hotels_data, restaurants_data, activity_data_new, transport_data, guides_data = load_data()

    # Store data globally
    global_data = {
        'activity_data': activity_data,
        'hotels_data': hotels_data,
        'restaurants_data': restaurants_data,
        'activity_data_new': activity_data_new,
        'transport_data': transport_data,
        'guides_data': guides_data  # Add this line
    }

# Run the Flask app in a separate thread
def run_flask_app():
    app.run()

# Start the system
initialize_system()

# Start Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

# Get the public URL
public_url = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print(f"Travel Itinerary Planner is now available at: {public_url.public_url}")

# Open the app in a new tab
display(HTML(f'<script>window.open("{public_url.public_url}", "_blank");</script>'))